<a href="https://colab.research.google.com/github/maxseats/2023_data_public/blob/main/whisper_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 모델 테스트 해보기
- 데이터셋 불러와서 평가지표(CER, WER)로 모델 점수 출력
- 파일로 저장
    - total.json : 모델 전체 평가
    - 각 모델 폴더 : 해당 모델 평가 과정 로그 기록
- 사용법 참고 : https://koogmlcks.notion.site/32f5ab9b703047d3a6002770ddf65e15?pvs=4

## 사전 작업

In [1]:
from google.colab import drive

# 구글드라이브 마운트
drive.mount('/content/drive')

!pip install nlptutti

import nlptutti as metrics
from transformers import pipeline
import json
import os
import time
import gdown
import gc
import torch

# 데이터셋 다운로드
gdown.download(id="12xNoD53zFqnkYYyeKm_box2gFR0WRCjb", output="dataset.zip", quiet=False)

# unzip
os.system("unzip dataset.zip")
os.system("rm dataset.zip")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Downloading...
From: https://drive.google.com/uc?id=12xNoD53zFqnkYYyeKm_box2gFR0WRCjb
To: /content/dataset.zip
100%|██████████| 6.22M/6.22M [00:00<00:00, 22.5MB/s]


0

## 사용자 설정 변수

In [17]:
#----- 모델 이름 설정 -----

'''
완료
"SungBeom/whisper-small-ko", "openai/whisper-large-v3", "steja/whisper-small-korean", "INo0121/whisper-base-ko-callvoice",
"seastar105/whisper-medium-ko-zeroth", "p4b/whisper-small-ko-fl-v2", "TheoJo/whisper-tiny-ko"
"biscayan/distil-whisper-large-v2-ko", "byoussef/whisper-large-v2-Ko", 'haseong8012/whisper-small-ko2'
'Gummybear05/whisper-small-ko-E50_Yspeed', 'jiwon65/whisper-small_korean-zeroth', 'aiegoo/whisper-tiny-ko'



진행 중



진행 예정
'gcasey2/whisper-large-v3-ko-en-v2',
'MSKim/korean-whisper',
'''

model_names = ['gcasey2/whisper-large-v3-ko-en-v2']

#----- 모델 이름 설정 -----


data_num = 75   # 데이터 개수
test_log_path = "/content/drive/MyDrive/STT_test/test_log"    # 테스트 결과 및 로그 저장위치
data_directory = "discord_dataset"                            # 데이터셋 폴더 지정

## 테스트 & 결과 기록

In [ ]:
# 모델 별 테스트 파이프라인 실행
for model_name in model_names:

    start_time = time.time()    # 시작 시간 기록

    # 평균 계산용
    CER_total = 0.0
    WER_total = 0.0

    # 모델 폴더 생성 및 로그파일 폴더 지정
    model_log_dir = os.path.join(test_log_path, model_name)
    os.makedirs(model_log_dir, exist_ok=True)
    log_file_path = os.path.join(model_log_dir, "log.txt")


    with open(log_file_path, 'w', encoding='utf-8') as log_file:


        # GPU 사용을 위해 device=0 설정
        device = 0 if torch.cuda.is_available() else -1
        pipe = pipeline("automatic-speech-recognition", model=model_name, device=device)   # STT 파이프라인

        for ii in range(data_num):

            i=ii+1   # 현재 파일 번호
            print(i, "번째 데이터:")
            log_file.write(f"{i} 번째 데이터:\n")

            sample = data_directory + "/" + "{:03d}".format(i) + ".mp3"    # 음성파일 경로

            result = pipe(sample, return_timestamps=False)

            preds = result["text"]  # STT 예측 문자열
            target_path = data_directory + "/" + "{:03d}".format(i) + ".txt" # 텍스트파일 경로


            # 파일 열기
            with open(target_path, 'r', encoding='utf-8') as file:
                # 파일 내용 읽기
                target = file.read()

            print("예측 : ", result["text"])
            print("정답 : ", target)
            log_file.write(f"예측 : {preds}\n")
            log_file.write(f"정답 : {target}\n")

            # CER 출력
            cer_result = metrics.get_cer(target, preds)

            cer_substitutions = cer_result['substitutions']
            cer_deletions = cer_result['deletions']
            cer_insertions = cer_result['insertions']
            # prints: [cer, substitutions, deletions, insertions] -> [CER = 0 / 34, S = 0, D = 0, I = 0]
            CER_total += cer_result['cer']
            print("CER, S, D, I : ", cer_result['cer'], cer_substitutions, cer_deletions, cer_insertions)
            log_file.write(f"CER, S, D, I : {cer_result['cer']}, {cer_substitutions}, {cer_deletions}, {cer_insertions}\n")


            # WER 출력
            wer_result = metrics.get_wer(target, preds)

            wer_substitutions = wer_result['substitutions']
            wer_deletions = wer_result['deletions']
            wer_insertions = wer_result['insertions']
            # prints: [wer, substitutions, deletions, insertions] -> [WER =  2 / 4, S = 1, D = 1, I = 0]
            WER_total += wer_result['wer']
            print("WER, S, D, I : ", wer_result['wer'], wer_substitutions, wer_deletions, wer_insertions)
            print()
            log_file.write(f"WER, S, D, I : {wer_result['wer']}, {wer_substitutions}, {wer_deletions}, {wer_insertions}\n\n")

            # 로그 버퍼에서 파일로 flush(중간 저장)
            log_file.flush()
            os.fsync(log_file.fileno())

    end_time = time.time()  # 종료 시간 기록
    elapsed_time = end_time - start_time    # 실행 시간

    # 시간, 분, 초 단위로 변환
    hours = int(elapsed_time // 3600)
    minutes = int((elapsed_time % 3600) // 60)
    seconds = int(elapsed_time % 60)


    print("현재 모델 : ", model_name)
    print("CER 평균 : ", CER_total / data_num)
    print("WER 평균 : ", WER_total / data_num)
    print("실행시간 : ", "{:02d}시간 {:02d}분 {:02d}초".format(hours, minutes, seconds))

    # 데이터 딕셔너리 생성
    data = {
        "model_name": model_name,
        "CER_mean": CER_total / data_num,
        "WER_mean": WER_total / data_num,
        "running_time" : "{:02d}:{:02d}:{:02d}".format(hours, minutes, seconds)
    }


    # 기존 데이터 읽기(없으면 빈 리스트)
    try:
        with open(test_log_path + "/total_result.json", "r", encoding="utf-8") as file:
            data_list = json.load(file)
    except FileNotFoundError:
        data_list = []

    # 새 데이터 추가
    data_list.append(data)

    # CER_mean, WER_mean을 기준으로 오름차순 정렬
    sorted_data = sorted(data_list, key=lambda x: (x['CER_mean'], x['WER_mean']))

    # 정렬된 데이터를 파일로 저장
    with open(test_log_path + "/total_result.json", "w", encoding="utf-8") as file:
        json.dump(sorted_data, file, ensure_ascii=False, indent=4)

    # 파이프라인 사용 후 메모리 해제
    del pipe
    gc.collect()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


1 번째 데이터:
예측 :  일단 녹음은 하고 있거든요
정답 :  일단 녹음은 하고 있거든요?
CER, S, D, I :  1.0 11 0 18
WER, S, D, I :  1.0 4 0 0

2 번째 데이터:
예측 :  핸드폰은 제가 핸드폰이 내 노트부보다 용량이 많아
정답 :  핸드폰은 제가 핸드폰이 내 내 노트북보다 용량이 많아.
CER, S, D, I :  1.0 22 0 28
WER, S, D, I :  1.0 7 1 0

3 번째 데이터:
예측 :  그래서 일단은 일단 지금부터 콜러가도 한번 보고 어
정답 :  그래서 일단은 일단 지금부터 클로바 노트 한번 보고 어
CER, S, D, I :  1.0 22 0 29
WER, S, D, I :  1.0 8 1 0

4 번째 데이터:
예측 :  클러버 노트를 썼을 때 불편함을 그냥 뭐 고요면서 같은 거 잘 못 알아봤고 그런 거 빼고 빼고 뭐 느꼈거든요 그리고 바라질 구분도 아마 그런 노터가 해줄 거야
정답 :  클로바 노트를 썼을 때 불편함을 그냥 뭐 고유명사 같은 거 잘 못 알아먹고 그런 거 빼고는 못 느꼈거든요? 그리고 발화자 구분도 아마 클로바 노트가 해줄 거야.
CER, S, D, I :  1.0 62 0 85
WER, S, D, I :  1.0 26 0 1

5 번째 데이터:
예측 :  아라자고 살아지더라
정답 :  발화자 구분 잘해주더라.
CER, S, D, I :  1.0 10 0 9
WER, S, D, I :  1.0 2 1 0

6 번째 데이터:
예측 :  어 그러면은 저희 ai 허브 가서 그건 봐야겠다
정답 :  어 아 그러면은 저희 AI 허브 가